In [2]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Multithreaded Architectures

Here is a "multithreaded" program.


In [3]:
compare([do_render_code("mv/matvec.c"), do_render_code("mv/matvec_threaded.c")])

## Simultaneous Multithreading (SMT)

Let's take a look how the threaded program would work on an SMT core -- we use `taskset` to fix both threads on the same physical core

In [17]:
! make -C  mv clean all
! echo ",IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branch_misses_pred_rate,branch_misses,branches" > stats.csv
! echo -n "single_thread," >> stats.csv
! taskset -c 11 ./mv/matvec 16384
! echo -n "two_threads_smt," >> stats.csv
! taskset -c 10,11 ./mv/matvec_threaded 16384
display_df_mono(render_csv("stats.csv"))
!cp stats.csv matvec_SMT.csv

make: Entering directory '/nfshome/htseng/courses/CS203/demo/multiprocessor/mv'
rm -f matvec matvec_threaded
cc -DHAVE_LINUX_PERF_EVENT_H -g -O3 -pthread -w  matvec.c perfstats.c -o matvec
cc -DHAVE_LINUX_PERF_EVENT_H -g -O3 -pthread -w  matvec_threaded.c perfstats.c -o matvec_threaded
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/multiprocessor/mv'
41845914.000000,41745652
41845914.000000,41745652


,index,Unnamed: 0,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branch_misses_pred_rate,branch_misses,branches
0,0,single_thread,1209818708,697212534,0.576295,0.195128,0.136046,0.022973,6182058,269099836,0.000220,29548,134553663
1,1,two_threads_smt,1223453853,861137192,0.703858,0.102878,0.088592,0.037068,10183024,274711733,0.000301,41221,136760027


In [20]:
! echo "IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branch_misses_pred_rate,branch_misses,branches" > stats.csv
! taskset -c 10,11 ./mv/matvec 16384 & taskset -c 10,11 ./mv/matvec 16384
display_df_mono(render_csv("stats.csv"))
!cp stats.csv two_identical_smt.csv

41845914.000000,41745652
41845914.000000,41745652


,index,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branch_misses_pred_rate,branch_misses,branches
0,0,1209896492,854456887,0.706223,0.193938,0.165712,0.041192,11085678,269120892,0.000135,18211,134569839
1,1,1209862303,838631563,0.693163,0.194475,0.163093,0.044397,11946815,269092124,0.000239,32185,134569308


## Chip Multiprocessors (CMP)

Let's try run them on separate cores.

In [12]:
! echo "IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branch_misses_pred_rate,branch_misses,branches" > stats.csv
! taskset -c 10 ./mv/matvec 16384 & taskset -c 0 ./mv/matvec 16384
display_df_mono(render_csv("stats.csv"))
!cp stats.csv two_identical_cmp.csv

41845914.000000,41745652
41845914.000000,41745652


,index,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branch_misses_pred_rate,branch_misses,branches
0,0,1209572512,821513599,0.679177,0.196386,0.161334,0.123834,33315625,269034891,0.000259,34806,134504489
1,1,1209131234,764965011,0.632657,0.196491,0.150309,0.021838,5871380,268858600,0.000252,33850,134432382


In [ ]:
## SMT vs CMP

In [16]:
! echo "IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branch_misses_pred_rate,branch_misses,branches" > stats.csv
! taskset -c 10,11 ./mv/matvec 16384 & taskset -c 10,11 ../OoO_programming/popcounts/popcount_D
display_df_mono(render_csv("stats.csv"))
!cp stats.csv two_different_smt.csv

41845914.000000,41745652
Result: 32000010811


,index,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branch_misses_pred_rate,branch_misses,branches
0,0,1231305279,751622829,0.610428,0.196249,0.147505,0.021718,6037217,277978070,0.000290,40017,138025812
1,1,80059284366,24343352043,0.304067,0.195932,4.769642,0.000055,1102599,20022200024,0.000062,62476,1010194714
